In [ ]:
import pandas as pd
import numpy as np
import PyUber

In [ ]:
sql = r"""SELECT 
          a0.lot7 AS lot7
         ,a0.devrevstep AS devrevstep
         ,(SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) AS last_pass
         ,a0.rework_latest_flag AS rework_latest_flag
         ,a0.lot_rework_flag AS rework_flag
         ,a1.entity AS entity
         ,a1.ceid AS ceid
         ,a2.monitor_set_name AS monitor_set_name
         ,To_Char(a2.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS data_collect_date
         ,a2.area AS area
         ,a2.monitor_process AS monitor_process
         ,a2.status AS mon_set_status
         ,a2.spc_data_id AS spc_data_id
         ,a2.violation_flag AS violation_flag
         ,a2.latest_flag AS mon_set_latest_flag
         ,a5.chart_point_seq AS chart_point_seq
         ,a5.value AS chart_value
         ,a5.process_chamber AS process_chamber
         ,a5.wafer AS chart_wafer
         ,a5.latest_flag AS chart_pt_latest_flag
         ,a5.status AS chart_pt_status
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_category AS spc_chart_category
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a10.lo_control_lmt AS lo_control_lmt
         ,a10.target AS target
         ,a10.up_control_lmt AS up_control_lmt
         ,a2.test_name AS test_name
         ,a0.operation AS spc_operation
         ,a5.wafer3 AS chart_wafer3
         ,a2.monitor_type AS monitor_type
         ,a3.parameter_class AS parameter_class
         ,a9.chart_parameter AS chart_parameter
         ,a9.chart_on AS chart_on
         ,a3.measurement_set_name AS measurement_set_name
         ,a11.
FROM 
P_SPC_LOT a0
LEFT JOIN P_SPC_ENTITY a1 ON a1.spcs_id = a0.spcs_id AND a1.entity_sequence=1
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a0.spcs_id AND a2.data_collection_time = a0.data_collection_time
INNER JOIN P_SPC_MEASUREMENT_SET a3 ON a3.spcs_id = a2.spcs_id
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
LEFT JOIN P_SPC_CHART a9 ON a9.chart_id = a5.chart_id
LEFT JOIN P_SPC_CHART_LIMIT a10 ON a10.chart_id = a5.chart_id AND a10.limit_id = a5.limit_id
LEFT JOIN P_SPC_CHARTPOINT_MEASUREMENT a11 ON a11.sps_it == a11.sps_id and a11.measurement_set_name == a4.measurement_set_name
WHERE
              a0.lot7 = 'L210060' 
 AND      (SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) = 'Y' 
 AND      a0.rework_latest_flag = 'Y' 
 AND      a3.valid_flag <> 'I' 
 AND      a5.latest_flag = 'Y' 
 AND      a3.latest_flag = 'Y' 
 AND      a5.value is not NULL"""

In [ ]:
sql2 = """
SELECT 
          a0.lot7 AS lot7
         ,a0.lot_type AS lot_type
         ,a0.lot_virtual_line AS lot_virtual_line
         ,a0.operation AS spc_operation
         ,a0.devrevstep AS devrevstep
         ,To_Char(a0.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS lot_data_collect_date
         ,a1.entity AS entity
         ,To_Char(a1.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS entity_data_collect_date
         ,a1.entity_sequence AS entity_sequence
         ,a2.monitor_set_name AS monitor_set_name
         ,a3.measurement_set_name AS measurement_set_name
         ,a2.test_name AS test_name
         ,a2.spc_data_id AS spc_data_id
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_category AS spc_chart_category
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a5.value AS chart_value
FROM 
P_SPC_LOT a0
LEFT JOIN P_SPC_ENTITY a1 ON a1.spcs_id = a0.spcs_id AND a1.entity_sequence=1
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a0.spcs_id AND a2.data_collection_time = a0.data_collection_time
INNER JOIN P_SPC_MEASUREMENT_SET a3 ON a3.spcs_id = a2.spcs_id
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
WHERE
              (SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) = 'Y' 
 AND      a3.latest_flag = 'Y' 
 AND      a5.latest_flag = 'Y' 
 AND      a0.data_collection_time >= (SYSDATE - 5/24) 
 AND      a0.lot Like 'L%' """

In [ ]:
con = PyUber.connect("F32_PROD_XEUS")

In [ ]:
df = pd.read_sql(sql, con)

In [ ]:
df = df[df['MONITOR_TYPE'] == "WIP MONITOR"]


In [ ]:
df['CHART_WAFER3'].value_counts()/df.shape[0]

In [ ]:
dfnl = df[df['CHART_WAFER3'].isna()]

In [ ]:
dfnl = dfnl[dfnl['CHART_PT_STATUS'] == 'V']

In [ ]:
dfnl['colnames'] = dfnl['SPC_OPERATION'] + '`' + dfnl['TEST_NAME'] + '`' + dfnl['MONITOR_SET_NAME'] + '`' + dfnl['MEASUREMENT_SET_NAME'] + '`' + dfnl['SPC_CHART_CATEGORY'] + '`' + dfnl['SPC_CHART_SUBSET'] + '`' + dfnl['CHART_TYPE'] + '`' + dfnl['PARAMETER_CLASS'] + '`' + dfnl['CHART_ON']
#dfnl.pivot(index=['LOT7'], columns='colnames', values = 'CHART_VALUE')

In [ ]:
sub = dfnl.groupby(['LOT7', 'colnames']).size()

In [ ]:
r = sub[sub > 1].reset_index()
#sub[sub['count']>1].head()

In [ ]:
dup = dfnl[dfnl['colnames'].isin(r['colnames'].unique())]

In [ ]:
dup['CHART_VALUE']

In [ ]:
dup['colnames'].unique().size, dfnl['colnames'].unique().size

In [ ]:
m = dup.iloc[1] != dup.iloc[0]
print(m)

In [ ]:
dup.iloc[1][m], dup.iloc[0][m]

In [ ]:
df = pd.read_sql("select * from P_SPC_CHARTPOINT_MEASUREMENT LIMIT 10", con)

In [ ]:
df.dtypes